# Analyzing course evaluations
This notebook provides sample [EDSL](https://pypi.org/project/edsl/) code for using a language model to analyze course evaluations. The analysis is designed as a survey of questions about a set of evaluations that we prompt an AI agent to answer, using a language model to generate the responses as a dataset.

[EDSL is an open-source library](https://github.com/expectedparrot/edsl) for simulating surveys, experiments and other research with AI agents and large language models. 
Before running the code below, please ensure that you have [installed the EDSL library](https://docs.expectedparrot.com/en/latest/installation.html) and either [activated remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) from your [Coop account](https://docs.expectedparrot.com/en/latest/coop.html) or [stored API keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for the language models that you want to use with EDSL. Please also see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started.

## Create questions
We start by creating questions about a set of course evaluations for an agent to answer. EDSL comes with a [variety of question types](https://docs.expectedparrot.com/en/latest/questions.html) that we can choose from based on the form of the response that we want to get back from a model (multiple choice, linear scale, checkbox, free text, numerical, dictionary, etc.). We can use a `{{ placeholder }}` in the question texts to parameterize them with each evaluation. This allows us to create different "scenarios" of the questions that we can administer at once.

We start by importing some question types and composing questions in the relevant templates (see [examples of all types](https://docs.expectedparrot.com/en/latest/questions.html#question-type-classes) in the docs):

In [1]:
from edsl import QuestionList, QuestionMultipleChoice

In [2]:
q_sentiment = QuestionMultipleChoice(
    question_name="sentiment",
    question_text="What is the overall sentiment of this evaluation: {{ evaluation }}",
    question_options=["Positive", "Neutral", "Negative"],
)

q_themes = QuestionList(
    question_name="themes",
    question_text="Summarize the key points of this evaluation: {{ evaluation }}",
    max_list_items=3,  # Optional
)

q_improvements = QuestionList(
    question_name="improvements",
    question_text="Identify areas for improvement based on this evaluation: {{ evaluation }}",
    max_list_items=3,
)

## Construct a survey
Next we combine our questions into a survey. This allows us to administer the questions asynchronously (by default), or according to any desired [survey logic or rules](https://docs.expectedparrot.com/en/latest/surveys.html) that we want to add, such as skip/stop rules or giving an agent "memories" of other questions in the survey. Here we create a simple asynchronous survey by passing the list of questions to a `Survey` object:

In [3]:
from edsl import Survey

survey = Survey(questions=[q_sentiment, q_themes, q_improvements])

## Select data for review
Next we identify the data to be analyzed. Here we use some mock evaluations for an Econ 101 course stored as a list of texts:

In [4]:
evaluations = [
    "I found the course very engaging and informative. The professor did an excellent job breaking down complex concepts, making them accessible to those of us new to economics. However, the pace was a bit fast, and I sometimes struggled to keep up with the weekly readings.",
    "This class was a struggle for me. The material felt dry and difficult to connect with real-world applications, which I think could have made it more interesting. More examples from current events would definitely have helped spark my interest.",
    "Excellent introductory course! The professor was enthusiastic and always willing to offer extra help during office hours. The interactive lectures and the practical assignments made the theory much more digestible and engaging.",
    "As someone with a strong background in math, I appreciated the analytical rigor of this course. However, I wish there had been more discussions that connected the theories we learned to everyday economic issues. It felt a bit isolated from practical realities at times.",
    "I enjoyed the course, especially the group projects, which were both challenging and rewarding. It was great to apply economic concepts to solve real-life problems. I did feel, however, that the feedback on assignments could be more detailed to help us understand our mistakes.",
    "The course content was well-organized, but the lectures were somewhat monotonous and hard to follow. I would suggest incorporating more visual aids and maybe some guest lectures from industry professionals to liven up the sessions.",
    "This was my favorite class this semester! The mix of theory and case studies was perfect, and the exams were fair. I also really appreciated the diversity of perspectives we explored in class, especially in terms of global economic policies.",
    "I found the textbook to be overly complex for an introductory course. It often used jargon that hadn't been explained in lectures, which was confusing. Simpler reading materials or more explanatory lectures would make a big difference for newcomers to economics.",
    "The professor was knowledgeable and clearly passionate about economics, but I felt the course relied too heavily on tests rather than more creative forms of assessment. More varied assignments would make the course more accessible to students with different learning styles.",
    "This class was a solid introduction to economics, though it leaned heavily on theoretical aspects. I would have liked more opportunities to discuss the real-world implications of economic theories, which I believe would enhance understanding and retention of the material.",
]

## Add data to the questions
Next we create a `ScenarioList` with a `Scenario` containing a key/value for each evaluation that we will add to the questions when we run the survey. EDSL provides [methods for generating scenarios from many data sources](https://docs.expectedparrot.com/en/latest/scenarios.html) (PDFs, CSVs, images, tables, dicts, etc.); here we import a list and match the key to our question texts placeholder:

In [5]:
from edsl import ScenarioList

scenarios = ScenarioList.from_list("evaluation", evaluations)

## Design AI agents
Next we design agents with relevant traits and personas for a language model to use in answering the questions. This can be useful if we want to compare responses among different audiences. We do this by passing dictionaries of `traits` to `Agent` objects. We can also choose whether to give an agent additional instructions for answering the survey (independent of individual question texts). Please see documentation for more [details and example code for creating agents to use with surveys](https://docs.expectedparrot.com/en/latest/agents.html).

Here we create a persona for the professor of the course and pass it some special instructions:

In [6]:
from edsl import Agent

persona = "You are a professor reviewing student evaluations for your recent Econ 101 course."
instruction = "Be very specific and constructive in providing feedback and suggestions."

agent = Agent(traits={"persona": persona}, instruction=instruction)

## Select language models
[EDSL works with many popular language models](https://docs.expectedparrot.com/en/latest/language_models.html) that we can use to generate responses for our survey. We can see a current list of all available models:

In [7]:
from edsl import Model

In [8]:
Model.available() 

/Users/a16174/edsl/edsl/inference_services/AvailableModelFetcher.py:139: UserWarning: No models found for service ollama
  warnings.warn(f"No models found for service {service_name}")


,Model Name,Service Name
0,Austism/chronos-hermes-13b-v2,deep_infra
1,Gryphe/MythoMax-L2-13b,deep_infra
2,Qwen/Qwen2-72B-Instruct,deep_infra
3,Qwen/Qwen2-7B-Instruct,deep_infra
4,Qwen/Qwen2.5-72B-Instruct,deep_infra
5,Sao10K/L3-70B-Euryale-v2.1,deep_infra
6,Sao10K/L3.1-70B-Euryale-v2.2,deep_infra
7,google/gemma-2-27b-it,deep_infra
8,google/gemma-2-9b-it,deep_infra
9,lizpreciatior/lzlv_70b_fp16_hf,deep_infra


We select models to use with a survey by creating `Model` objects for them. The default model is GPT 4 Preview, meaning that EDSL will use it to run our survey if we do not specify a different model. Here's we'll specify that GPT 4o should be used:

In [9]:
model = Model("gemini-1.5-flash")
model

,key,value
0,model,gemini-1.5-flash
1,parameters:temperature,0.500000
2,parameters:topP,1
3,parameters:topK,1
4,parameters:maxOutputTokens,2048
5,parameters:stopSequences,[]


## Run the survey
Next we add the scenarios and agent to the survey, and then run it with the specified model. This will generate a dataset of `Results` that we can store and begin analyzing:

In [10]:
results = survey.by(scenarios).by(agent).by(model).run()

Job UUID,93073e7c-6837-4ef3-87ca-2ebd7fcf2831
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/93073e7c-6837-4ef3-87ca-2ebd7fcf2831
Error Report URL,None
Results UUID,07846ba6-a514-401f-a8ac-bc02f3e81783
Results URL,None


## Analyzing results
EDSL comes with [built-in methods for analyzing results](https://docs.expectedparrot.com/en/latest/results.html) in data tables, dataframes, SQL queries and other formats. We can print a list of all the components that can be accessed:

In [11]:
results.columns

,0
0,agent.agent_index
1,agent.agent_instruction
2,agent.agent_name
3,agent.persona
4,answer.improvements
5,answer.sentiment
6,answer.themes
7,cache_used.improvements_cache_used
8,cache_used.sentiment_cache_used
9,cache_used.themes_cache_used


Here we select just the responses to the questions and display them in a table:

In [12]:
results.select("sentiment", "themes", "themes_generated_tokens", "improvements")

,answer.sentiment,answer.themes,generated_tokens.themes_generated_tokens,answer.improvements
0,Positive,"['Engaging and informative course content', 'Effective explanation of complex concepts', 'Fast pace and challenging readings']","[""Engaging and informative course content"", ""Effective explanation of complex concepts"", ""Fast pace and challenging readings""] The student found the course to be highly engaging and well-taught, but noted the pace was too fast, impacting their ability to keep up with the readings. These three points capture the essence of the feedback.","['Adjust the pace of lectures and incorporate more opportunities for student engagement.', 'Provide more structured guidance on tackling the weekly readings, perhaps by suggesting specific questions to consider.', 'Consider offering supplemental resources, such as online tutorials or practice problems, to reinforce concepts.']"
1,Negative,"['Student found the material dry and difficult', 'Lack of real-world connection and examples cited', 'Desire for more current event examples to increase engagement']","[""Student found the material dry and difficult"", ""Lack of real-world connection and examples cited"", ""Desire for more current event examples to increase engagement""] The three points directly address the student's criticisms and are concise. They avoid subjective interpretations and focus on the student's stated concerns.","['Incorporate more real-world examples and case studies', 'Connect course material to current events', 'Develop more engaging teaching methods to improve student comprehension']"
2,Positive,"['Excellent instruction and enthusiasm', 'Effective interactive lectures and practical assignments', 'Readily available extra help during office hours']","[""Excellent instruction and enthusiasm"", ""Effective interactive lectures and practical assignments"", ""Readily available extra help during office hours""] The three points chosen directly reflect the student's positive feedback on teaching quality, course structure, and accessibility of the instructor. They are concise and capture the essence of the evaluation.","['Consider incorporating more diverse case studies', 'Explore alternative assessment methods to gauge deeper understanding beyond practical assignments', 'Enhance online resources and accessibility for students with varying learning styles']"
3,Neutral,"['Appreciated analytical rigor', 'Desired more real-world application discussions', 'Felt disconnect between theory and practice']","[""Appreciated analytical rigor"", ""Desired more real-world application discussions"", ""Felt disconnect between theory and practice""] The student's feedback highlights both a positive aspect (strong math foundation) and a key area for improvement (linking theory to practical examples). The three points concisely capture the essence of the evaluation.","['Incorporate real-world case studies into lectures', 'Dedicate class time for discussions linking theoretical concepts to current events', 'Assign application-based problem sets or projects']"
4,Positive,"['Enjoyed the course and group projects', 'Project application of concepts was valuable', 'Feedback on assignments needs improvement']","[""Enjoyed the course and group projects"", ""Project application of concepts was valuable"", ""Feedback on assignments needs improvement""] The student enjoyed the course material and found the group projects particularly engaging and beneficial for applying economic concepts. However, they clearly identified a need for more detailed feedback on assignments to improve learning. This summarizes the key points concisely and addresses both positive and negative aspects.","['Enhance feedback detail on assignments', 'Consider providing more specific rubric criteria', 'Explore alternative assessment methods to complement group projects']"
5,Neutral,"['Well-organized course content', 'Monotonous and hard-to-follow lectures', 'Suggestions for improvement: visual aids and

We can do a quick tally of the sentiments:

In [13]:
results.select("sentiment").tally()

,answer.sentiment,count
0,Positive,4
1,Neutral,4
2,Negative,2


We can also transform the results into a dataframe:

In [14]:
df = results.to_pandas()
df.head()

,answer.themes,answer.sentiment,answer.improvements,scenario.scenario_index,scenario.evaluation,agent.agent_index,agent.persona,agent.agent_name,agent.agent_instruction,model.topP,...,question_type.improvements_question_type,comment.themes_comment,comment.improvements_comment,comment.sentiment_comment,generated_tokens.themes_generated_tokens,generated_tokens.improvements_generated_tokens,generated_tokens.sentiment_generated_tokens,cache_used.themes_cache_used,cache_used.sentiment_cache_used,cache_used.improvements_cache_used
0,"['Engaging and informative course content', 'E...",Positive,['Adjust the pace of lectures and incorporate ...,0,I found the course very engaging and informati...,0,You are a professor reviewing student evaluati...,Agent_10,Be very specific and constructive in providing...,1,...,list,The student found the course to be highly enga...,The student's feedback highlights two key area...,The student clearly expresses overall positive...,"[""Engaging and informative course content"", ""E...","[""Adjust the pace of lectures and incorporate ...",Positive\n\nThe student clearly expresses over...,False,False,False
1,['Student found the material dry and difficult...,Negative,['Incorporate more real-world examples and cas...,1,This class was a struggle for me. The material...,0,You are a professor reviewing student evaluati...,Agent_10,Be very specific and constructive in providing...,1,...,list,The three points directly address the student'...,These suggestions directly address the student...,The student explicitly states the class was a ...,"[""Student found the material dry and difficult...","[""Incorporate more real-world examples and cas...",Negative\n\nThe student explicitly states the ...,False,False,False
2,"['Excellent instruction and enthusiasm', 'Effe...",Positive,['Consider incorporating more diverse case stu...,2,Excellent introductory course! The professor w...,0,You are a professor reviewing student evaluati...,Agent_10,Be very specific and constructive in providing...,1,...,list,The three points chosen directly reflect the s...,"While the feedback is largely positive, the s...",The student uses overwhelmingly positive langu...,"[""Excellent instruction and enthusiasm"", ""Effe...","[""Consider incorporating more diverse case stu...",Positive\n\nThe student uses overwhelmingly po...,False,False,False
3,"['Appreciated analytical rigor', 'Desired more...",Neutral,['Incorporate real-world case studies into lec...,3,"As someone with a strong background in math, I...",0,You are a professor reviewing student evaluati...,Agent_10,Be very specific and constructive in providing...,1,...,list,The student's feedback highlights both a posit...,The student's feedback highlights a disconnect...,The evaluation contains both positive and nega...,"[""Appreciated analytical rigor"", ""Desired more...","[""Incorporate real-world case studies into lec...",Neutral\n\nThe evaluation contains both positi...,False,False,False
4,"['Enjoyed the course and group projects', 'Pro...",Positive,"['Enhance feedback detail on assignments', 'Co...",4,"I enjoyed the course, especially the group pro...",0,You are a professor reviewing student evaluati...,Agent_10,Be very specific and constructive in providing...,1,...,list,The student enjoyed the course material and fo...,The student enjoyed the course and group proje...,The student clearly enjoyed the course and fou...,"[""Enjoyed the course and group projects"", ""Pro...","[""Enhance feedback detail on assignments"", ""Co...",Positive\n\nThe student clearly enjoyed the co...,False,False,False


We can also use pandas methods by first converting:

In [15]:
df_sentiment = results.to_pandas()["answer.sentiment"]
df_sentiment.value_counts()

answer.sentiment
Positive    4
Neutral     4
Negative    2
Name: count, dtype: int64

## Use responses to construct new questions
We can use the responses to our initial questions to construct more questions about the texts. For example, we can prompt a model to condense the individual lists of themes and areas for improvement into short lists, and then use the new lists to quantify the topics across the set of evaluations.

Here we take the lists of themes in each evaluation, flatten them into a (duplicative) list, and then create a new question prompting a model to condense it for us:

In [16]:
results.select("themes", "themes_generated_tokens")

,answer.themes,generated_tokens.themes_generated_tokens
0,"['Engaging and informative course content', 'Effective explanation of complex concepts', 'Fast pace and challenging readings']","[""Engaging and informative course content"", ""Effective explanation of complex concepts"", ""Fast pace and challenging readings""] The student found the course to be highly engaging and well-taught, but noted the pace was too fast, impacting their ability to keep up with the readings. These three points capture the essence of the feedback."
1,"['Student found the material dry and difficult', 'Lack of real-world connection and examples cited', 'Desire for more current event examples to increase engagement']","[""Student found the material dry and difficult"", ""Lack of real-world connection and examples cited"", ""Desire for more current event examples to increase engagement""] The three points directly address the student's criticisms and are concise. They avoid subjective interpretations and focus on the student's stated concerns."
2,"['Excellent instruction and enthusiasm', 'Effective interactive lectures and practical assignments', 'Readily available extra help during office hours']","[""Excellent instruction and enthusiasm"", ""Effective interactive lectures and practical assignments"", ""Readily available extra help during office hours""] The three points chosen directly reflect the student's positive feedback on teaching quality, course structure, and accessibility of the instructor. They are concise and capture the essence of the evaluation."
3,"['Appreciated analytical rigor', 'Desired more real-world application discussions', 'Felt disconnect between theory and practice']","[""Appreciated analytical rigor"", ""Desired more real-world application discussions"", ""Felt disconnect between theory and practice""] The student's feedback highlights both a positive aspect (strong math foundation) and a key area for improvement (linking theory to practical examples). The three points concisely capture the essence of the evaluation."
4,"['Enjoyed the course and group projects', 'Project application of concepts was valuable', 'Feedback on assignments needs improvement']","[""Enjoyed the course and group projects"", ""Project application of concepts was valuable"", ""Feedback on assignments needs improvement""] The student enjoyed the course material and found the group projects particularly engaging and beneficial for applying economic concepts. However, they clearly identified a need for more detailed feedback on assignments to improve learning. This summarizes the key points concisely and addresses both positive and negative aspects."
5,"['Well-organized course content', 'Monotonous and hard-to-follow lectures', 'Suggestions for improvement: visual aids and guest lecturers']","[""Well-organized course content"", ""Monotonous and hard-to-follow lectures"", ""Suggestions for improvement: visual aids and guest lecturers""] The summary accurately reflects the student's main points: positive feedback on organization, negative feedback on lecture style, and concrete suggestions for improvement. The three points are concise and capture the essence of the evaluation."
6,"['Enjoyed the balance of theory and case studies', 'Exams perceived as fair', 'Appreciated diverse perspectives on global economic policies']","[""Enjoyed the balance of theory and case studies"", ""Exams perceived as fair"", ""Appreciated diverse perspectives on global economic policies""] The student's positive feedback highlights three key aspects: the course structure, assessment fairness, and the inclusion of diverse viewpoints. These are valuable points for course improvement and demonstrate student engagement."
7,"['Textbook complexity', 'Unexplained jargon', 'Need for simpler materials or more explanatory lectures']","[""Textbook complexity"", ""Unexplained jargon"", ""Need for simpler materials or more explanatory lectures""] The student clearly identifies three interconne

In [17]:
themes = results.select("themes").to_list(flatten = True)

Next we construct a question to condense the list into a new list:

In [18]:
q_condensed_themes = QuestionList(
    question_name="condensed_themes",
    question_text="""Combine the following list of themes extracted from the evaluations 
    into a consolidated, non-redundant list: """
    + ", ".join(themes),
    max_list_items=10,
)

Now we run the question and select the new list. Note that we can choose whether we want to use the agent for this question by not adding it to the question when we run it:

In [19]:
condensed_themes = q_condensed_themes.run().select("condensed_themes").to_list()[0]
condensed_themes

Job UUID,71d589e6-f842-4c72-a3f3-e45634370220
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/71d589e6-f842-4c72-a3f3-e45634370220
Error Report URL,None
Results UUID,2a2c0493-2364-4a50-b15d-32cec16cabc3
Results URL,None


['Engaging and informative course content',
 'Effective explanation of complex concepts',
 'Need for more real-world examples and applications',
 'Excellent instruction and enthusiasm',
 'Desire for more varied and accessible learning materials',
 'Well-organized course content',
 'Constructive feedback on assignments',
 'Appreciated analytical rigor and diverse perspectives',
 'Suggestions for improvement: visual aids and guest lecturers',
 'Fair exams and valuable project applications']

Now we can create a question to identify all the themes in the list that appear in each evaluation (our new list becomes the list of answer options):

In [20]:
from edsl import QuestionCheckBox

q_themes_list = QuestionCheckBox(
    question_name="themes_list",
    question_text="Select all of the themes that are mentioned in this evaluation: {{ evaluation }}",
    question_options=condensed_themes,
)

Here we run the question and show a table listing all the themes for each evaluation in the results:

In [21]:
themes_lists = q_themes_list.by(scenarios).by(agent).run()
themes_lists.select("evaluation", "themes_list")

Job UUID,141752af-2db7-4896-9445-83d272105d1d
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/141752af-2db7-4896-9445-83d272105d1d
Error Report URL,None
Results UUID,7a9b833b-b952-431c-b7b0-9ab8321b6a86
Results URL,None


,scenario.evaluation,answer.themes_list
0,"I found the course very engaging and informative. The professor did an excellent job breaking down complex concepts, making them accessible to those of us new to economics. However, the pace was a bit fast, and I sometimes struggled to keep up with the weekly readings.","['Engaging and informative course content', 'Effective explanation of complex concepts']"
1,"This class was a struggle for me. The material felt dry and difficult to connect with real-world applications, which I think could have made it more interesting. More examples from current events would definitely have helped spark my interest.","['Need for more real-world examples and applications', 'Desire for more varied and accessible learning materials']"
2,Excellent introductory course! The professor was enthusiastic and always willing to offer extra help during office hours. The interactive lectures and the practical assignments made the theory much more digestible and engaging.,"['Engaging and informative course content', 'Excellent instruction and enthusiasm']"
3,"As someone with a strong background in math, I appreciated the analytical rigor of this course. However, I wish there had been more discussions that connected the theories we learned to everyday economic issues. It felt a bit isolated from practical realities at times.","['Need for more real-world examples and applications', 'Appreciated analytical rigor and diverse perspectives']"
4,"I enjoyed the course, especially the group projects, which were both challenging and rewarding. It was great to apply economic concepts to solve real-life problems. I did feel, however, that the feedback on assignments could be more detailed to help us understand our mistakes.","['Engaging and informative course content', 'Constructive feedback on assignments', 'Fair exams and valuable project applications']"
5,"The course content was well-organized, but the lectures were somewhat monotonous and hard to follow. I would suggest incorporating more visual aids and maybe some guest lectures from industry professionals to liven up the sessions.","['Well-organized course content', 'Suggestions for improvement: visual aids and guest lecturers']"
6,"This was my favorite class this semester! The mix of theory and case studies was perfect, and the exams were fair. I also really appreciated the diversity of perspectives we explored in class, especially in terms of global economic policies.","['Engaging and informative course content', 'Appreciated analytical rigor and diverse perspectives', 'Fair exams and valuable project applications']"
7,"I found the textbook to be overly complex for an introductory course. It often used jargon that hadn't been explained in lectures, which was confusing. Simpler reading materials or more explanatory lectures would make a big difference for newcomers to economics.","['Effective explanation of complex concepts', 'Desire for more varied and accessible learning materials']"
8,"The professor was knowledgeable and clearly passionate about economics, but I felt the course relied too heavily on tests rather than more creative forms of assessment. More varied assignments would make the course more accessible to students with different learning styles.","['Excellent instruction and enthusiasm', 'Desire for more varied and accessible learning materials']"
9,"This class was a solid introduction to economics, though it leaned heavily on theoretical aspects. I would have liked more opportunities to discuss the real-world implications of economic theories, which I believe would enhance understanding and retention of the material.",['Need for more real-world examples and applications']


In [22]:
wide_evaluation_themes = themes_lists.select("evaluation", "themes_list").to_scenario_list().expand("themes_list").rename({"themes_list": "theme"})
wide_evaluation_themes

,evaluation,theme
0,"I found the course very engaging and informative. The professor did an excellent job breaking down complex concepts, making them accessible to those of us new to economics. However, the pace was a bit fast, and I sometimes struggled to keep up with the weekly readings.",Engaging and informative course content
1,"I found the course very engaging and informative. The professor did an excellent job breaking down complex concepts, making them accessible to those of us new to economics. However, the pace was a bit fast, and I sometimes struggled to keep up with the weekly readings.",Effective explanation of complex concepts
2,"This class was a struggle for me. The material felt dry and difficult to connect with real-world applications, which I think could have made it more interesting. More examples from current events would definitely have helped spark my interest.",Need for more real-world examples and applications
3,"This class was a struggle for me. The material felt dry and difficult to connect with real-world applications, which I think could have made it more interesting. More examples from current events would definitely have helped spark my interest.",Desire for more varied and accessible learning materials
4,Excellent introductory course! The professor was enthusiastic and always willing to offer extra help during office hours. The interactive lectures and the practical assignments made the theory much more digestible and engaging.,Engaging and informative course content
5,Excellent introductory course! The professor was enthusiastic and always willing to offer extra help during office hours. The interactive lectures and the practical assignments made the theory much more digestible and engaging.,Excellent instruction and enthusiasm
6,"As someone with a strong background in math, I appreciated the analytical rigor of this course. However, I wish there had been more discussions that connected the theories we learned to everyday economic issues. It felt a bit isolated from practical realities at times.",Need for more real-world examples and applications
7,"As someone with a strong background in math, I appreciated the analytical rigor of this course. However, I wish there had been more discussions that connected the theories we learned to everyday economic issues. It felt a bit isolated from practical realities at times.",Appreciated analytical rigor and diverse perspectives
8,"I enjoyed the course, especially the group projects, which were both challenging and rewarding. It was great to apply economic concepts to solve real-life problems. I did feel, however, that the feedback on assignments could be more detailed to help us understand our mistakes.",Engaging and informative course content
9,"I enjoyed the course, especially the group projects, which were both challenging and rewarding. It was great to apply economic concepts to solve real-life problems. I did feel, however, that the feedback on assignments could be more detailed to help us understand our mistakes.",Constructive feedback on assignments


In [23]:
wide_evaluation_themes.tally("theme")

,theme,count
0,Engaging and informative course content,4
1,Need for more real-world examples and applications,3
2,Desire for more varied and accessible learning materials,3
3,Effective explanation of complex concepts,2
4,Excellent instruction and enthusiasm,2
5,Appreciated analytical rigor and diverse perspectives,2
6,Fair exams and valuable project applications,2
7,Constructive feedback on assignments,1
8,Well-organized course content,1
9,Suggestions for improvement: visual aids and guest lecturers,1


We can do the same thing with the areas of improvement:

In [24]:
improvements = results.select("improvements").to_list(flatten=True)
improvements

,0
0,Adjust the pace of lectures and incorporate more opportunities for student engagement.
1,"Provide more structured guidance on tackling the weekly readings, perhaps by suggesting specific questions to consider."
2,"Consider offering supplemental resources, such as online tutorials or practice problems, to reinforce concepts."
3,Incorporate more real-world examples and case studies
4,Connect course material to current events
5,Develop more engaging teaching methods to improve student comprehension
6,Consider incorporating more diverse case studies
7,Explore alternative assessment methods to gauge deeper understanding beyond practical assignments
8,Enhance online resources and accessibility for students with varying learning styles
9,Incorporate real-world case studies into lectures


In [25]:
q_condensed_improvements = QuestionList(
    question_name="condensed_improvements",
    question_text="""Combine the following list of areas for improvement from the evaluations 
    into a consolidated, non-redundant list: """
    + ", ".join(improvements),
    max_list_items=10,
)

In [26]:
condensed_improvements = (
    q_condensed_improvements.run().select("condensed_improvements").to_list()[0]
)
condensed_improvements

Job UUID,d636dc50-f043-4419-b5b9-187654d66197
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/d636dc50-f043-4419-b5b9-187654d66197
Error Report URL,None
Results UUID,94a10c71-6c06-4b9a-94c8-e15e5806ff4e
Results URL,None


['Adjust the pace of lectures and enhance engagement through interactive methods',
 'Provide structured guidance and supplemental resources for weekly readings',
 'Incorporate real-world examples, case studies, and current events into the curriculum',
 'Explore diverse and alternative assessment methods beyond traditional exams',
 'Enhance feedback and provide detailed rubric criteria for assignments',
 'Improve clarity and engagement in lecture delivery with visual aids',
 'Develop assignments catering to varied learning styles and preferences',
 'Incorporate industry insights through guest lecturers and practical examples',
 'Designate time for discussions linking theory to real-world applications',
 'Simplify and supplement textbook materials for clarity and comprehension']

In [27]:
q_improvements_list = QuestionCheckBox(
    question_name="improvements_list",
    question_text="Select all of the improvements that are mentioned in this evaluation: {{ evaluation }}",
    question_options=condensed_improvements,
)

In [28]:
improvements_lists = q_improvements_list.by(scenarios).by(agent).run()
improvements_lists.select("evaluation", "improvements_list")

Job UUID,97580159-903a-4555-986a-1529654b41fa
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/97580159-903a-4555-986a-1529654b41fa
Error Report URL,None
Results UUID,71193b91-5f0c-4c22-9b79-e9ed3d0c97ce
Results URL,None


,scenario.evaluation,answer.improvements_list
0,"I found the course very engaging and informative. The professor did an excellent job breaking down complex concepts, making them accessible to those of us new to economics. However, the pace was a bit fast, and I sometimes struggled to keep up with the weekly readings.","['Adjust the pace of lectures and enhance engagement through interactive methods', 'Provide structured guidance and supplemental resources for weekly readings']"
1,"This class was a struggle for me. The material felt dry and difficult to connect with real-world applications, which I think could have made it more interesting. More examples from current events would definitely have helped spark my interest.","['Incorporate real-world examples, case studies, and current events into the curriculum', 'Designate time for discussions linking theory to real-world applications']"
2,Excellent introductory course! The professor was enthusiastic and always willing to offer extra help during office hours. The interactive lectures and the practical assignments made the theory much more digestible and engaging.,[]
3,"As someone with a strong background in math, I appreciated the analytical rigor of this course. However, I wish there had been more discussions that connected the theories we learned to everyday economic issues. It felt a bit isolated from practical realities at times.","['Incorporate real-world examples, case studies, and current events into the curriculum', 'Designate time for discussions linking theory to real-world applications']"
4,"I enjoyed the course, especially the group projects, which were both challenging and rewarding. It was great to apply economic concepts to solve real-life problems. I did feel, however, that the feedback on assignments could be more detailed to help us understand our mistakes.",['Enhance feedback and provide detailed rubric criteria for assignments']
5,"The course content was well-organized, but the lectures were somewhat monotonous and hard to follow. I would suggest incorporating more visual aids and maybe some guest lectures from industry professionals to liven up the sessions.","['Improve clarity and engagement in lecture delivery with visual aids', 'Incorporate industry insights through guest lecturers and practical examples']"
6,"This was my favorite class this semester! The mix of theory and case studies was perfect, and the exams were fair. I also really appreciated the diversity of perspectives we explored in class, especially in terms of global economic policies.",[]
7,"I found the textbook to be overly complex for an introductory course. It often used jargon that hadn't been explained in lectures, which was confusing. Simpler reading materials or more explanatory lectures would make a big difference for newcomers to economics.","['Provide structured guidance and supplemental resources for weekly readings', 'Simplify and supplement textbook materials for clarity and comprehension']"
8,"The professor was knowledgeable and clearly passionate about economics, but I felt the course relied too heavily on tests rather than more creative forms of assessment. More varied assignments would make the course more accessible to students with different learning styles.","['Explore diverse and alternative assessment methods beyond traditional exams', 'Develop assignments catering to varied learning styles and preferences']"
9,"This class was a solid introduction to economics, though it leaned heavily on theoretical aspects. I would have liked more opportunities to discuss the real-world implications of economic theories, which I believe would enhance understanding and retention of the material.","['Incorporate real-world examples, case studies, and current events into the curriculum', 'Designate time for discussions linking theory to real-world applications']"


In [29]:
wide_themes = (
    improvements_lists
    .select("evaluation", "improvements_list")
    .to_scenario_list()
    .expand("improvements_list")
    .rename({"improvements_list": "theme"})
)

In [30]:
wide_themes.tally("theme")

,theme,count
0,"Incorporate real-world examples, case studies, and current events into the curriculum",3
1,Designate time for discussions linking theory to real-world applications,3
2,Provide structured guidance and supplemental resources for weekly readings,2
3,Adjust the pace of lectures and enhance engagement through interactive methods,1
4,Enhance feedback and provide detailed rubric criteria for assignments,1
5,Improve clarity and engagement in lecture delivery with visual aids,1
6,Incorporate industry insights through guest lecturers and practical examples,1
7,Simplify and supplement textbook materials for clarity and comprehension,1
8,Explore diverse and alternative assessment methods beyond traditional exams,1
9,Develop assignments catering to varied learning styles and preferences,1


## Other examples
Please check out the [EDSL Docs](https://docs.expectedparrot.com/en/latest/index.html) for examples of other methods and templates for use cases, and [join our Discord channel](https://discord.com/invite/mxAYkjfy9m) to ask questions and with other users!

## Posting to the Coop
The [Coop](https://www.expectedparrot.com/content/explore) is a platform for creating, storing and sharing LLM-based research.
It is fully integrated with EDSL and accessible from your workspace or Coop account page.
Learn more about [creating an account](https://www.expectedparrot.com/login) and [using the Coop](https://docs.expectedparrot.com/en/latest/coop.html).

We can post any EDSL object to the Coop by calling the `push()` method on it, including this notebook:

In [34]:
from edsl import Notebook

In [35]:
n = Notebook(path = "analyze_evaluations.ipynb")

In [36]:
info = n.push(description = "Example code for analyzing course evaluations", visibility = "public")
info

{'description': 'Example code for analyzing course evaluations',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/4e2808a7-2b4e-4603-8ad2-6e1a65989ccb',
 'uuid': '4e2808a7-2b4e-4603-8ad2-6e1a65989ccb',
 'version': '0.1.40.dev1',
 'visibility': 'public'}

To update an object:

In [37]:
n = Notebook(path = "analyze_evaluations.ipynb") # resave

In [38]:
n.patch(uuid = info["uuid"], value = n)

{'status': 'success'}